In [1]:
import json
import pandas as pd
import ollama
import requests
import ast
import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
resumes = pd.read_csv('datasets/Resume.csv')

In [3]:
jobs = pd.read_csv('datasets/dice_com-job_us_sample.csv')

In [4]:
resumes

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR
...,...,...,...,...
2479,99416532,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2480,24589765,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","<div class=""fontsize fontface vmargins hmargin...",AVIATION
2481,31605080,GEEK SQUAD AGENT Professional...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2482,21190805,PROGRAM DIRECTOR / OFFICE MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION


In [5]:
jobs

,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id
0,https://www.dice.com/jobs/detail/AUTOMATION-TE...,"Digital Intelligence Systems, LLC","C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...",Looking for Selenium engineers...must have sol...,Dice Id : 10110693,"Atlanta, GA",AUTOMATION TEST ENGINEER,1 hour ago,Telecommuting not available|Travel not required,NaN,SEE BELOW,418ff92580b270ef4e7c14f0ddfc36b4
1,https://www.dice.com/jobs/detail/Information-S...,University of Chicago/IT Services,Full Time,The University of Chicago has a rapidly growin...,Dice Id : 10114469,"Chicago, IL",Information Security Engineer,1 week ago,Telecommuting not available|Travel not required,NaN,"linux/unix, network monitoring, incident respo...",8aec88cba08d53da65ab99cf20f6f9d9
2,https://www.dice.com/jobs/detail/Business-Solu...,"Galaxy Systems, Inc.",Full Time,"GalaxE.SolutionsEvery day, our solutions affec...",Dice Id : CXGALXYS,"Schaumburg, IL",Business Solutions Architect,2 weeks ago,Telecommuting not available|Travel not required,NaN,"Enterprise Solutions Architecture, business in...",46baa1f69ac07779274bcd90b85d9a72
3,https://www.dice.com/jobs/detail/Java-Develope...,TransTech LLC,Full Time,Java DeveloperFull-time/direct-hireBolingbrook...,Dice Id : 10113627,"Bolingbrook, IL","Java Developer (mid level)- FT- GREAT culture,...",2 weeks ago,Telecommuting not available|Travel not required,NaN,Please see job description,3941b2f206ae0f900c4fba4ac0b18719
4,https://www.dice.com/jobs/detail/DevOps-Engine...,Matrix Resources,Full Time,Midtown based high tech firm has an immediate ...,Dice Id : matrixga,"Atlanta, GA",DevOps Engineer,48 minutes ago,Telecommuting not available|Travel not required,NaN,"Configuration Management, Developer, Linux, Ma...",45efa1f6bc65acc32bbbb953a1ed13b7
...,...,...,...,...,...,...,...,...,...,...,...,...
21995,https://www.dice.com/jobs/detail/Web-Designer-...,IAC Publishing,Full Time,Company Description We are searching for a ta...,Dice Id : 10112803,"Oakland, CA",Web Designer,3 weeks ago,Telecommuting not available|Travel not required,NaN,"UI/UX mobile apps, interaction design, digital...",86e27ce6b7e631e55d69d142c7d43df2
21996,https://www.dice.com/jobs/detail/Senior-Front-...,Omega Solutions Inc,Full Time,CONTACT - priya@omegasolutioninc.com / 408-45...,Dice Id : 10289500,"San Francisco, CA",Senior Front End Web Developer - Full Time at ...,3 weeks ago,Telecommuting not available|Travel not required,NaN,"JavaScript, HTML5, CSS3, Bootstrap, AJAX, Reac...",4287c7ee3317ccf1edd76e238cf8e584
21997,https://www.dice.com/jobs/detail/QA-Analyst-Sa...,San Francisco Health Plan,Full Time,Do you take pride in your work knowing that th...,Dice Id : 10115761,"San Francisco, CA",QA Analyst,2 weeks ago,Telecommuting not available|Travel not required,NaN,"SDLC, ALM, SQL, T-SQL, RedGate, Team Foundatio...",d7512f0181d69f83f96db38cd77a4d08
21998,https://www.dice.com/jobs/detail/Tech-Lead%252...,IAC Publishing,Full Time,Company Description What We Can Offer YouAs th...,Dice Id : 10112803,"Oakland, CA",Tech Lead-Full Stack,2 weeks ago,Telecommuting not available|Travel not required,NaN,"Python, Ruby, Go, Clojure, Java, NoSQL-Databas...",ec375268b494b3bcbed1635d64226112


In [6]:
resumes['Category'].value_counts()

Category
INFORMATION-TECHNOLOGY    120
BUSINESS-DEVELOPMENT      120
ADVOCATE                  118
CHEF                      118
ENGINEERING               118
ACCOUNTANT                118
FINANCE                   118
FITNESS                   117
AVIATION                  117
SALES                     116
BANKING                   115
HEALTHCARE                115
CONSULTANT                115
CONSTRUCTION              112
PUBLIC-RELATIONS          111
HR                        110
DESIGNER                  107
ARTS                      103
TEACHER                   102
APPAREL                    97
DIGITAL-MEDIA              96
AGRICULTURE                63
AUTOMOBILE                 36
BPO                        22
Name: count, dtype: int64

In [7]:
resumes[resumes['Category']=='INFORMATION-TECHNOLOGY']

,ID,Resume_str,Resume_html,Category
217,36856210,INFORMATION TECHNOLOGY Summar...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY
218,21780877,INFORMATION TECHNOLOGY SPECIALIST\tGS...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY
219,33241454,INFORMATION TECHNOLOGY SUPERVISOR ...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY
220,25990239,INFORMATION TECHNOLOGY INSTRUCTOR ...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY
221,16899268,INFORMATION TECHNOLOGY MANAGER/ANALYS...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY
...,...,...,...,...
332,27485716,CORPORATE PROJECT MANAGER Caree...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY
333,26480367,IT TECHNOLOGY SPECIALIST Profes...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY
334,81761658,IT MANAGER Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY
335,79541391,SUBJECT MATTER EXPERT (INFORMATION TE...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY


In [8]:
def get_response(pompt, model="mistral:instruct"):
    response = ollama.generate(model=model, prompt=pompt)
    return response

In [9]:
def get_prompt(description, doc_type):
    if doc_type not in ['resume', 'job']:
        return "Invalid type"
    
    prompt = f"""Extract the skills from the following {doc_type}, return a comma-separated list. 
Do not include education and certifications.
Do not return the same skill more than one even if it's mentiond multiple times. 
Do not include years of experience or seniority levels, only the skill names.
Return only JSON array of strings.

    {description}"""

    return prompt

In [10]:
def get_skills(document, doc_type):
    prompt = get_prompt(document, doc_type)
    response = get_response(prompt)
    skills = ast.literal_eval(response['response'].strip())
    return skills

In [11]:
def get_embedding(text):
    response = ollama.embeddings(model="nomic-embed-text", prompt=text)
    return response

In [12]:
resumes.iloc[50]['Resume_str']

"         HR ASSISTANT/PAYROLL         Highlights        Paychex, Mastertax, ABRA, Time Star, InfoTronics, Store Master, Dominion, QuickBooks\nCeridian, PeopleSoft, SAP, Microsoft Word, Excel and PowerPoint, Crystal Reports              Experience      HR Assistant/Payroll    Company Name   －   City  ,   State      Human Resources professional offering over 15 years of combined HR and payroll\n          experience in both manufacturing and academic settings\nAccurate and timely preparation of employee payroll, payroll tax information, and\n          employee benefits administration\nHighly effective employer representation in unemployment claims and hearings.         01/2014   to   01/2014     Payroll & Benefits Coordinator    Company Name   －   City  ,   State      Process weekly payroll for 100 hourly and salaried Associates\n    Garnishment, Levy and Child Support processing\n    Compile weekly labor hours      \n    Enter new hire information into HRIS/Payroll system\n    Assist wi

In [13]:
get_skills(resumes.iloc[50]['Resume_str'], 'resume')

['ABRA',
 'academic',
 'Benefits',
 'benefits administration',
 'billing',
 'bi',
 'Ceridian',
 'conflict resolution',
 'CRYSTAL REPORTS',
 'HRIS',
 'Human Resources',
 'HR',
 'insurance',
 'Process invoices',
 'local government',
 'materials',
 'meetings',
 'Excel',
 'PowerPoint',
 'Microsoft Word',
 'Payroll',
 'PeopleSoft',
 'performance reviews',
 'personnel',
 'policies',
 'QuickBooks',
 'reporting',
 'safety',
 'sales',
 'SAP',
 'spreadsheets',
 'tax',
 'tax returns',
 'written']

In [ ]:
get_skills(jobs.iloc[50]['jobdescription'], 'job')

In [ ]:
resumes

In [ ]:
resumes['extracted_skills'] = None

In [ ]:
for index, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    prompt = get_prompt(row['Resume_str'], 'resume')
    response = get_response(prompt)
    skills = ast.literal_eval(response['response'].strip())
    resumes.at[index, 'extracted_skills'] = skills

In [ ]:
def process_row(row):
    prompt = get_prompt(row['Resume_str'], 'resume')
    response = get_response(prompt)
    skills = ast.literal_eval(response['response'].strip())
    return row.name, skills 

In [ ]:
results = {}
with ThreadPoolExecutor(max_workers=5) as executor:  # Adjust max_workers as needed
    # Submit all tasks
    future_to_index = {executor.submit(process_row, row): row.name for index, row in resumes.iterrows()}
    
    # Process completed tasks with progress bar
    for future in tqdm.tqdm(as_completed(future_to_index), total=len(resumes)):
        index, skills = future.result()
        resumes.at[index, 'extracted_skills'] = skills

In [ ]:
resumes['extracted_skills'] = results

In [ ]:
esco = pd.read_csv('datasets/skills_en.csv')

In [ ]:
esco['preferredLabel'].tolist()

In [ ]:
esco_skills = esco['preferredLabel'].tolist()

In [ ]:
esco_skills[:10]

In [ ]:
response = ollama.embed(model="nomic-embed-text", input=esco_skills)

In [ ]:
esco['embeddings'] = response['embeddings']

In [ ]:
esco.to_parquet('processed/esco_skills.parquet', index=False)